In [120]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

np.set_printoptions(suppress=True,)

In [121]:
header  = ["Click", "Weekday", "Hour", "Timestamp", "Log Type", "User ID", "User‐Agent", "IP", "Region", "City", "Ad Exchange", "Domain", "URL", "Anonymous URL ID", "Ad slot ID", "Ad slot width", "Ad slot height", "Ad slot visibility", "Ad slot format", "Ad slot floor price (RMB/CPM)", "Creative ID", "Key Page URL", "Advertiser ID", "User Tags"]
print(alist)

['Click', 'Weekday', 'Hour', 'Timestamp', 'Log Type', 'User ID', 'User\xe2\x80\x90Agent', 'IP', 'Region', 'City', 'Ad Exchange', 'Domain', 'URL', 'Anonymous URL ID', 'Ad slot ID', 'Ad slot width', 'Ad slot height', 'Ad slot visibility', 'Ad slot format', 'Ad slot floor price (RMB/CPM)', 'Creative ID', 'Key Page URL', 'Advertiser ID', 'User Tags']


In [122]:
trainfile = 'trainhead100000.txt'
testfile = 'testhead.txt'

train = pd.read_csv(trainfile, header = None, sep = '\t', names = header)
test = pd.read_csv(testfile, header = None, sep = '\t', names = header[1:])

### Show head lines of files
# print(train.head())
#print(test.head())

### output to csv file
# train.head().to_csv('out.csv', sep = '\t')

In [123]:
### Check the structure of input files
# print(train.describe())
# print(train.shape)

In [131]:
### Logistic Regression
mytrain = train.ix[:, [1,2,8,9,10,15,16,17,18]]
mytest = test.ix[:, [0,1,7,8,9,14,15,16,17]]
#print(mytest.describe())

lr = LogisticRegression(C = 1e+5)
lr.fit(mytrain, train["Click"])

### Predict class label for test samples
# result = lr.predict(mytest)

### Predict probability for each class label
result = lr.predict_proba(mytest)

print(result[:100, 1])

[ 0.1688  0.0004  0.2825  0.0004  0.0004  0.0011  0.3264  0.3006  0.156
  0.0397  0.5979  0.2698  0.1299  0.1972  0.11    0.211   0.1089  0.2664
  0.001   0.3373  0.1132  0.2006  0.001   0.1118  0.0005  0.3227  0.1726
  0.0004  0.0002  0.0003  0.0853  0.0003  0.2496  0.293   0.1973  0.0002
  0.126   0.0349  0.307   0.1213  0.0002  0.1669  0.1396  0.0006  0.4349
  0.1632  0.0004  0.1593  0.395   0.2945  0.0002  0.2828  0.2801  0.072
  0.2859  0.4154  0.3349  0.3131  0.1672  0.2298  0.1587  0.1672  0.0004
  0.0004  0.0003  0.001   0.1426  0.0941  0.154   0.2537  0.0803  0.0624
  0.2208  0.0778  0.2732  0.1351  0.3729  0.0009  0.1405  0.1665  0.0005
  0.1547  0.1793  0.1628  0.0297  0.2099  0.0003  0.1465  0.0005  0.0005
  0.0003  0.0001  0.0008  0.164   0.0001  0.0001  0.1041  0.3124  0.155
  0.0531]
